In [1]:
!pip install -U -q PyDrive

In [2]:
import sqlite3
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
!mkdir data

In [6]:
file_id = '1KiOvhsdjJqaUCLJa5adZXEtQ_72s8Eb6'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data/checking-logs.sqlite')

In [8]:
conn = sqlite3.connect('data/checking-logs.sqlite')

In [9]:
test_results = pd.io.sql.read_sql(
    """
    select time, avg(diff) as avg_diff
    from
        (
            select
                uid,
                cast((julianday(test.checker_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 as integer) as diff,
                case
                    when test.checker_ts < test.first_view_ts then 'before'
                    else 'after'
                end as time
            from test left join deadlines on test.labname = deadlines.labs
            where not test.labname = 'project1'
        )
    where uid in
        (
            select uid
            from
                (
                    select
                        uid,
                        case
                            when test.checker_ts < test.first_view_ts then 'before'
                            else 'after'
                        end as time
                    from test left join deadlines on test.labname = deadlines.labs
                    where not test.labname = 'project1'
                )
            group by uid
            having count(distinct time) = 2
        )
    group by time
    """,
    conn
)

In [11]:
test_results

,time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [14]:
control_results = pd.io.sql.read_sql(
    """
    select time, avg(diff) as avg_diff
    from
        (
            select
                uid,
                cast((julianday(control.checker_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 as integer) as diff,
                case
                    when control.checker_ts < control.first_view_ts then 'before'
                    else 'after'
                end as time
            from control left join deadlines on control.labname = deadlines.labs
            where not control.labname = 'project1'
        )
    where uid in
        (
            select uid
            from
                (
                    select
                        uid,
                        case
                            when control.checker_ts < control.first_view_ts then 'before'
                            else 'after'
                        end as time
                    from control left join deadlines on control.labname = deadlines.labs
                    where not control.labname = 'project1'
                )
            group by uid
            having count(distinct time) = 2
        )
    group by time
    """,
    conn
)

In [15]:
control_results

,time,avg_diff
0,after,-117.636364
1,before,-99.464286


In [17]:
conn.close()